# Task 5

In [6]:
import pandas as pd

# Load the CSV file
data = pd.read_csv('D:\\Study\\Parul University\\PUStudy\\4th sem\\#Data Science With Python\\#Internship\\Task\\5\\heart.csv')

# Display the first few rows of the dataset
print(data.head())
print(data.info())
print(data.describe())


   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   52    1   0       125   212    0        1      168      0      1.0      2   
1   53    1   0       140   203    1        0      155      1      3.1      0   
2   70    1   0       145   174    0        1      125      1      2.6      0   
3   61    1   0       148   203    0        1      161      0      0.0      2   
4   62    0   0       138   294    1        1      106      0      1.9      1   

   ca  thal  target  
0   2     3       0  
1   0     3       0  
2   0     3       0  
3   1     3       0  
4   3     2       0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1025 non-null   int64  
 1   sex       1025 non-null   int64  
 2   cp        1025 non-null   int64  
 3   trestbps  1025 non-null   int64  
 4   chol      1025 non-null   int64  
 

In [7]:
# Create age groups
data['age_group'] = pd.cut(data['age'], bins=[0, 40, 50, 60, 100], labels=['<40', '40-50', '50-60', '60+'])

# Create cholesterol level categories
data['chol_category'] = pd.cut(data['chol'], bins=[0, 200, 239, 1000], labels=['normal', 'borderline', 'high'])

# Create interaction terms
data['age_chol'] = data['age'] * data['chol']
data['thalach_age'] = data['thalach'] * data['age']

# Display the first few rows of the dataset with the new features
print(data.head())


   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   52    1   0       125   212    0        1      168      0      1.0      2   
1   53    1   0       140   203    1        0      155      1      3.1      0   
2   70    1   0       145   174    0        1      125      1      2.6      0   
3   61    1   0       148   203    0        1      161      0      0.0      2   
4   62    0   0       138   294    1        1      106      0      1.9      1   

   ca  thal  target age_group chol_category  age_chol  thalach_age  
0   2     3       0     50-60    borderline     11024         8736  
1   0     3       0     50-60    borderline     10759         8215  
2   0     3       0       60+        normal     12180         8750  
3   1     3       0       60+    borderline     12383         9821  
4   3     2       0       60+          high     18228         6572  


In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Selecting relevant features for PCA
features = data.drop(columns=['target', 'age_group', 'chol_category'])
# Encode categorical features
features = pd.get_dummies(features, drop_first=True)

# Standardize the features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Apply PCA
pca = PCA(n_components=0.95)  # retain 95% of the variance
features_pca = pca.fit_transform(features_scaled)

# Check the explained variance ratio to determine the number of components
explained_variance = pca.explained_variance_ratio_
n_components = pca.n_components_

print(features_pca.shape)
print(explained_variance)
print(n_components)


(1025, 12)
[0.20889427 0.16830688 0.08840103 0.0812662  0.06882297 0.06670356
 0.0589318  0.05530184 0.05083943 0.0478213  0.0436604  0.03479448]
12


In [9]:
from sklearn.ensemble import RandomForestClassifier

# Prepare the data for the Random Forest model
X = pd.get_dummies(data.drop(columns=['target', 'age_group', 'chol_category']), drop_first=True)
y = data['target']

# Train a Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X, y)

# Get feature importances
feature_importances = rf_model.feature_importances_
important_features = pd.Series(feature_importances, index=X.columns).sort_values(ascending=False)

# Display the most important features
print(important_features.head(15))


cp             0.120862
ca             0.112540
thal           0.110047
oldpeak        0.090816
thalach        0.089553
age            0.071684
thalach_age    0.071066
age_chol       0.067411
exang          0.060699
chol           0.055705
trestbps       0.051452
slope          0.041440
sex            0.033022
restecg        0.016684
fbs            0.007018
dtype: float64


In [10]:
# Select the top 15 important features
top_features = important_features.head(15).index
X_top = X[top_features]

# Train a new Random Forest model using only the top features
rf_model_top = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_top.fit(X_top, y)

# Evaluate the new model
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf_model_top, X_top, y, cv=5)
print(f'Cross-validated scores: {scores}')
print(f'Mean score: {scores.mean()}')


Cross-validated scores: [1.         1.         1.         1.         0.98536585]
Mean score: 0.9970731707317073
